In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os

In [11]:
import pandas as pd
import numpy as np
import os
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [2]:
path = '../output/'

In [3]:
joined_data = pd.read_csv(path + 'data_btu_railbelt.csv')
joined_data = joined_data.drop(columns =['Unnamed: 0'])

In [4]:
joined_data.head()

,areasq_ft,height,osm_id,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,...,age,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020,FD_80,TD_80,age_yrs,annual_btu_persqft,annual_btu
0,4121.072347,0.314958,258262706,99586,9958,-2558.431424,1392.266167,-2515.636046,1460.472344,-2291.114094,...,2019.000000,-2599.099527,1370.239804,-2468.616556,1426.342060,-2560.558700,1346.086352,3.000000,NaN,NaN
1,22186.284252,6.400278,289169932,99586,9958,-2284.837221,1726.294096,-2238.464469,1787.345975,-2027.679677,...,1989.221571,-2355.672257,1703.871241,-2207.272135,1757.478646,-2260.374937,1687.665071,32.778429,NaN,NaN
2,2632.210905,2.592521,430515206,99586,9958,-2842.391776,1191.221087,-2811.422750,1245.677326,-2550.397921,...,2019.000000,-2893.268582,1175.813066,-2751.696418,1218.934420,-2822.483997,1152.172869,3.000000,NaN,NaN
3,5759.540437,1.128282,456366962,99586,9958,-2842.391776,1191.221087,-2811.422750,1245.677326,-2550.397921,...,2019.000000,-2893.268582,1175.813066,-2751.696418,1218.934420,-2822.483997,1152.172869,3.000000,NaN,NaN
4,3838.153050,2.705054,469671205,99586,9958,-2303.060885,1444.133000,-2272.443214,1497.505776,-2063.150925,...,2019.000000,-2352.630126,1426.974372,-2229.408088,1470.013202,-2284.109315,1407.918851,3.000000,NaN,NaN


# even split between fairbanks and anchorage

In [5]:
# Fairbanks
zip_fb = [99701, 99702, 99703, 99705, 99706, 99707, 99708, 99709, 99710, 
          99711, 99712, 99714, 99716, 99725, 99775, 99790]

# Anchorage
zip_an = [99501, 99502, 99503, 99504, 99505, 99506, 99507, 99508, 99509, 99510,
           99511, 99513, 99514, 99515, 99516, 99517, 99518, 99519, 99520, 99521,
           99522, 99523, 99524, 99529, 99530, 99540, 99599, 99695]

In [6]:
fairbanks_only = joined_data[joined_data['zip_code'].isin(zip_fb)]
anchorage_only = joined_data[joined_data['zip_code'].isin(zip_an)]

In [7]:
new_fairbanks = fairbanks_only.copy(deep=True)
num_samples = len(anchorage_only) - len(fairbanks_only)
new_fairbanks = new_fairbanks.append(fairbanks_only.sample(n = num_samples, replace = True))

/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/2440515283.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fairbanks = new_fairbanks.append(fairbanks_only.sample(n = num_samples, replace = True))


In [8]:
new_fairbanks.head()

,areasq_ft,height,osm_id,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,...,age,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020,FD_80,TD_80,age_yrs,annual_btu_persqft,annual_btu
1319,11824.985096,1.538692,876559385,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1987.799485,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,34.200515,127.423651,1.506783e+06
1320,10519.818412,1.081035,922721944,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,1985.000000,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,37.000000,130.205500,1.369738e+06
1321,5696.133192,1.217605,926237024,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,1988.397508,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,33.602492,126.829396,7.224371e+05
1322,4088.935206,0.456894,928743729,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1998.914122,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,23.085878,116.379037,4.758663e+05
1323,2646.478578,-0.455877,962093403,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1985.000000,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,37.000000,130.205500,3.445861e+05


In [9]:
# combined data with augemented fairbanks data
combined_fair_anch = new_fairbanks.copy(deep=True)
combined_fair_anch = combined_fair_anch.append(anchorage_only)

/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/3327099098.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_fair_anch = combined_fair_anch.append(anchorage_only)


In [10]:
combined_fair_anch.head()

,areasq_ft,height,osm_id,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,...,age,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020,FD_80,TD_80,age_yrs,annual_btu_persqft,annual_btu
1319,11824.985096,1.538692,876559385,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1987.799485,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,34.200515,127.423651,1.506783e+06
1320,10519.818412,1.081035,922721944,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,1985.000000,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,37.000000,130.205500,1.369738e+06
1321,5696.133192,1.217605,926237024,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,1988.397508,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,33.602492,126.829396,7.224371e+05
1322,4088.935206,0.456894,928743729,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1998.914122,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,23.085878,116.379037,4.758663e+05
1323,2646.478578,-0.455877,962093403,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1985.000000,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,37.000000,130.205500,3.445861e+05


In [12]:
combined_fair_anch.head()

,areasq_ft,height,osm_id,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,...,age,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020,FD_80,TD_80,age_yrs,annual_btu_persqft,annual_btu
1319,11824.985096,1.538692,876559385,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1987.799485,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,34.200515,127.423651,1.506783e+06
1320,10519.818412,1.081035,922721944,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,1985.000000,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,37.000000,130.205500,1.369738e+06
1321,5696.133192,1.217605,926237024,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,1988.397508,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,33.602492,126.829396,7.224371e+05
1322,4088.935206,0.456894,928743729,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1998.914122,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,23.085878,116.379037,4.758663e+05
1323,2646.478578,-0.455877,962093403,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,1985.000000,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,37.000000,130.205500,3.445861e+05


In [13]:
combined_fair_anch.columns

Index(['areasq_ft', 'height', 'osm_id', 'zip_code', 'zip_group',
       'FD_1981_2010', 'TD_1981_2010', 'FD_2000', 'TD_2000', 'FD_2010',
       'TD_2010', 'age', 'FD_90', 'TD_90', 'FD_recent_1991_2020',
       'TD_recent_1991_2020', 'FD_80', 'TD_80', 'age_yrs',
       'annual_btu_persqft', 'annual_btu'],
      dtype='object')

In [14]:
# change FD cols to be positive
combined_fair_anch['FD_1981_2010'] = abs(combined_fair_anch['FD_1981_2010'])
combined_fair_anch['FD_2000'] = abs(combined_fair_anch['FD_2000'])
combined_fair_anch['FD_2010'] = abs(combined_fair_anch['FD_2010'])
combined_fair_anch['FD_90'] = abs(combined_fair_anch['FD_90'])
combined_fair_anch['FD_80'] = abs(combined_fair_anch['FD_80'])

In [15]:
Xy_fairbanks = combined_fair_anch[combined_fair_anch['zip_code'].isin(zip_fb)]
Xy_anchorage = combined_fair_anch[combined_fair_anch['zip_code'].isin(zip_an)]

In [16]:
# Create numpy arrays for scikit learn 
features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group']
# features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

X_fair = np.array(Xy_fairbanks.drop(features_dropped, axis=1))
y_fair = np.array(Xy_fairbanks['annual_btu_persqft'])
y_fair = y_fair.reshape([y_fair.shape[0],1])

X_anch = np.array(Xy_anchorage.drop(features_dropped, axis=1))
y_anch = np.array(Xy_anchorage['annual_btu_persqft'])
y_anch = y_anch.reshape([y_anch.shape[0],1])

In [17]:
# standardize data

scaler = StandardScaler()
scaler.fit(np.vstack((X_fair, X_anch)))
X_fair = scaler.transform(X_fair)
X_anch = scaler.transform(X_anch)

scaler = StandardScaler()
scaler.fit(np.vstack((y_fair, y_anch)))
y_fair = scaler.transform(y_fair)
y_anch = scaler.transform(y_anch)



In [18]:
# Divide intro training and testing datasets, with a 30-70 ratio
X_train_anch, X_test_anch, y_train_anch, y_test_anch = train_test_split(X_anch, y_anch, test_size = 0.3, random_state = 1)

X_train_fair, X_test_fair, y_train_fair, y_test_fair = train_test_split(X_fair, y_fair, test_size = 0.3, random_state = 1)



In [19]:
# combine fairbanks and anchorage
X_train = np.vstack((X_train_anch, X_train_fair))
X_test = np.vstack((X_test_anch, X_test_fair))
y_train = np.vstack((y_train_anch, y_train_fair))
y_test = np.vstack((y_test_anch, y_test_fair))

In [20]:
with open(path + 'data_sampling/equal_anch_fair/x_train.npy', 'wb') as f:
    np.save(f, X_train)

    
with open(path + 'data_sampling/equal_anch_fair/x_test.npy', 'wb') as f:
    np.save(f, X_test)
    
    
with open(path + 'data_sampling/equal_anch_fair/y_train.npy', 'wb') as f:
    np.save(f, y_train)
    
with open(path + 'data_sampling/equal_anch_fair/y_test.npy', 'wb') as f:
    np.save(f, y_test)

# Building age

In [89]:
joined_data = pd.read_csv(path + 'data_btu_railbelt.csv')
joined_data = joined_data.drop(columns =['Unnamed: 0'])

In [90]:
# bin age
joined_data['year_rounded'] = abs(joined_data['age'].round() - 2020)

In [91]:
# change FD cols to be positive
joined_data['FD_1981_2010'] = abs(joined_data['FD_1981_2010'])
joined_data['FD_2000'] = abs(joined_data['FD_2000'])
joined_data['FD_2010'] = abs(joined_data['FD_2010'])
joined_data['FD_90'] = abs(joined_data['FD_90'])
joined_data['FD_80'] = abs(joined_data['FD_80'])

In [92]:
joined_data.head()

,areasq_ft,height,osm_id,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,...,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020,FD_80,TD_80,age_yrs,annual_btu_persqft,annual_btu,year_rounded
0,4121.072347,0.314958,258262706,99586,9958,2558.431424,1392.266167,2515.636046,1460.472344,2291.114094,...,2599.099527,1370.239804,-2468.616556,1426.342060,2560.558700,1346.086352,3.000000,NaN,NaN,1.0
1,22186.284252,6.400278,289169932,99586,9958,2284.837221,1726.294096,2238.464469,1787.345975,2027.679677,...,2355.672257,1703.871241,-2207.272135,1757.478646,2260.374937,1687.665071,32.778429,NaN,NaN,31.0
2,2632.210905,2.592521,430515206,99586,9958,2842.391776,1191.221087,2811.422750,1245.677326,2550.397921,...,2893.268582,1175.813066,-2751.696418,1218.934420,2822.483997,1152.172869,3.000000,NaN,NaN,1.0
3,5759.540437,1.128282,456366962,99586,9958,2842.391776,1191.221087,2811.422750,1245.677326,2550.397921,...,2893.268582,1175.813066,-2751.696418,1218.934420,2822.483997,1152.172869,3.000000,NaN,NaN,1.0
4,3838.153050,2.705054,469671205,99586,9958,2303.060885,1444.133000,2272.443214,1497.505776,2063.150925,...,2352.630126,1426.974372,-2229.408088,1470.013202,2284.109315,1407.918851,3.000000,NaN,NaN,1.0


In [93]:
joined_data_nonmissing = joined_data.dropna()

In [94]:
joined_data_nonmissing['year_rounded'].unique()

array([32., 35., 21.,  1., 26., 34., 28., 23., 29., 18., 25., 15., 10.,
       17., 19., 31., 12., 33., 22., 27., 20., 30.,  0.,  6., 24., 16.,
       11., 36., 14.,  8.,  5.,  7.,  9., 13.])

In [95]:
years_df_dict = {}
for year in joined_data_nonmissing['year_rounded'].unique():
    years_df_dict[year] = joined_data_nonmissing[joined_data_nonmissing['year_rounded'] == year]

In [96]:
max_count = 0
for year in years_df_dict.keys():
    #print('year: ', year)
    #print(len(years_df_dict[year]))
    
    if len(years_df_dict[year])> max_count:
        max_count = len(years_df_dict[year])

In [97]:
max_count

37180

In [98]:
# up and down sample to 5000
num_samples = 5000
sampled_dfs = {}
for year in years_df_dict.keys():
    current_df = years_df_dict[year]
    num_buildings = len(current_df)
    new_df = pd.DataFrame()
    
    if num_buildings > num_samples:
        # downsample
        new_df = new_df.append(current_df.sample(n=num_samples)) 
    elif num_buildings < num_samples:
        # upsample
        new_df = new_df.append(current_df)
        new_df = new_df.append(current_df.sample(n=num_samples - len(current_df), replace=True))
    else:
        new_df = current_df
    
    sampled_dfs[year] = new_df
    

/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/3495386210.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(current_df)
/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/3495386210.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(current_df.sample(n=num_samples - len(current_df), replace=True))
/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/3495386210.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(current_df.sample(n=num_samples))
/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/3495386210.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pand

In [99]:
# Create numpy arrays for scikit learn 
features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group', 'year_rounded']
# features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

X_list = []
y_list = []
for year in sampled_dfs.keys():
    df = sampled_dfs[year]
    X = np.array(df.drop(features_dropped, axis=1))
    y = np.array(df['annual_btu_persqft'])
    y = y.reshape([y.shape[0],1])
    
    X_list.append(X)
    y_list.append(y)
    

In [100]:
sampled_dfs[0].columns

Index(['areasq_ft', 'height', 'osm_id', 'zip_code', 'zip_group',
       'FD_1981_2010', 'TD_1981_2010', 'FD_2000', 'TD_2000', 'FD_2010',
       'TD_2010', 'age', 'FD_90', 'TD_90', 'FD_recent_1991_2020',
       'TD_recent_1991_2020', 'FD_80', 'TD_80', 'age_yrs',
       'annual_btu_persqft', 'annual_btu', 'year_rounded'],
      dtype='object')

In [101]:
# standardize data
scaler = StandardScaler()
scaler.fit(np.vstack((X_list)))
X_list = [scaler.transform(x_elem) for x_elem in X_list]


scaler = StandardScaler()
scaler.fit(np.vstack((y_list)))
y_list = [scaler.transform(y_elem) for y_elem in y_list]

In [102]:
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
for i in range(len(X_list)):
    X = X_list[i]
    y = y_list[i]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
    
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)


In [103]:
# combine ages
X_train = np.vstack((X_train_list))
X_test = np.vstack((X_test_list))
y_train = np.vstack((y_train_list))
y_test = np.vstack((y_test_list))

In [104]:
with open(path + 'data_sampling/equal_age/x_train.npy', 'wb') as f:
    np.save(f, X_train)

    
with open(path + 'data_sampling/equal_age/x_test.npy', 'wb') as f:
    np.save(f, X_test)
    
    
with open(path + 'data_sampling/equal_age/y_train.npy', 'wb') as f:
    np.save(f, y_train)
    
with open(path + 'data_sampling/equal_age/y_test.npy', 'wb') as f:
    np.save(f, y_test)

# equal sq ft-- unsure about how to choose the bins here

In [115]:
joined_data = pd.read_csv(path + 'data_btu_railbelt.csv')
joined_data = joined_data.drop(columns =['Unnamed: 0'])

In [116]:
joined_data_nonmissing = joined_data.dropna()

In [117]:
pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3)

[(0.994, 3.0], (5.0, 7.0], (3.0, 5.0], (3.0, 5.0], (5.0, 7.0], (0.994, 3.0]]
Categories (3, interval[float64, right]): [(0.994, 3.0] < (3.0, 5.0] < (5.0, 7.0]]

In [118]:
joined_data_nonmissing['sq_ft_bin'] = pd.qcut(joined_data_nonmissing['areasq_ft'], 50, labels = np.arange(50))


/var/folders/xc/3ndr1sy52yd9dpvk9qlffhdw0000gn/T/ipykernel_49522/1799674595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_data_nonmissing['sq_ft_bin'] = pd.qcut(joined_data_nonmissing['areasq_ft'], 50, labels = np.arange(50))


In [80]:
joined_data_nonmissing.head()

,areasq_ft,height,osm_id,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,...,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020,FD_80,TD_80,age_yrs,annual_btu_persqft,annual_btu,sq_ft_bin
1319,11824.985096,1.538692,876559385,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,34.200515,127.423651,1.506783e+06,48
1320,10519.818412,1.081035,922721944,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,37.000000,130.205500,1.369738e+06,48
1321,5696.133192,1.217605,926237024,99701,9970,-2698.087432,1893.504650,-2611.400742,1959.360701,-2342.077081,...,-2758.506492,1870.740203,-2570.661438,1912.035993,-2724.355061,1850.413047,33.602492,126.829396,7.224371e+05,46
1322,4088.935206,0.456894,928743729,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,23.085878,116.379037,4.758663e+05,44
1323,2646.478578,-0.455877,962093403,99701,9970,-2773.715598,1903.920607,-2678.494453,1970.666617,-2397.632758,...,-2837.759796,1882.265915,-2637.962336,1923.728242,-2804.892545,1858.829290,37.000000,130.205500,3.445861e+05,38


In [86]:
for i in range(50):
    print(len(joined_data_nonmissing[joined_data_nonmissing['sq_ft_bin'] == i]))
    print(joined_data_nonmissing[joined_data_nonmissing['sq_ft_bin'] ==i]['sq_ft_bin'].max())
    

1496
0
1496
1
1496
2
1495
3
1496
4
1496
5
1495
6
1496
7
1496
8
1496
9
1495
10
1496
11
1496
12
1495
13
1496
14
1496
15
1495
16
1496
17
1496
18
1496
19
1495
20
1496
21
1496
22
1495
23
1496
24
1496
25
1495
26
1496
27
1496
28
1496
29
1495
30
1496
31
1496
32
1495
33
1496
34
1496
35
1495
36
1496
37
1496
38
1496
39
1495
40
1496
41
1496
42
1495
43
1496
44
1496
45
1495
46
1496
47
1496
48
1496
49


In [ ]:
# # bin sq ft
# joined_data['year_rounded'] = abs(joined_data['age'].round() - 2020)

In [ ]:
# # change FD cols to be positive
# joined_data['FD_1981_2010'] = abs(joined_data['FD_1981_2010'])
# joined_data['FD_2000'] = abs(joined_data['FD_2000'])
# joined_data['FD_2010'] = abs(joined_data['FD_2010'])
# joined_data['FD_90'] = abs(joined_data['FD_90'])
# joined_data['FD_80'] = abs(joined_data['FD_80'])

In [ ]:
# joined_data_nonmissing['year_rounded'].unique()

In [ ]:
# years_df_dict = {}
# for year in joined_data_nonmissing['year_rounded'].unique():
#     years_df_dict[year] = joined_data_nonmissing[joined_data_nonmissing['year_rounded'] == year]

In [ ]:
# max_count = 0
# for year in years_df_dict.keys():
#     #print('year: ', year)
#     #print(len(years_df_dict[year]))
    
#     if len(years_df_dict[year])> max_count:
#         max_count = len(years_df_dict[year])

In [ ]:
# # up and down sample to 5000
# num_samples = 5000
# sampled_dfs = {}
# for year in years_df_dict.keys():
#     current_df = years_df_dict[year]
#     num_buildings = len(current_df)
#     new_df = pd.DataFrame()
    
#     if num_buildings > num_samples:
#         # downsample
#         new_df = new_df.append(current_df.sample(n=num_samples)) 
#     elif num_buildings < num_samples:
#         # upsample
#         new_df = new_df.append(current_df)
#         new_df = new_df.append(current_df.sample(n=num_samples - len(current_df), replace=True))
#     else:
#         new_df = current_df
    
#     sampled_dfs[year] = new_df
    

In [ ]:
# # Create numpy arrays for scikit learn 
# features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group', 'sq_ft_bin']
# # features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

# X_list = []
# y_list = []
# for year in sampled_dfs.keys():
#     df = sampled_dfs[year]
#     X = np.array(df.drop(features_dropped, axis=1))
#     y = np.array(df['annual_btu_persqft'])
#     y = y.reshape([y.shape[0],1])
    
#     X_list.append(X)
#     y_list.append(y)
    

In [ ]:
# # standardize data
# scaler = StandardScaler()
# scaler.fit(np.vstack((X_list)))
# X_list = [scaler.transform(x_elem) for x_elem in X_list]


# scaler = StandardScaler()
# scaler.fit(np.vstack((y_list)))
# y_list = [scaler.transform(y_elem) for y_elem in y_list]

In [ ]:
# X_train_list = []
# X_test_list = []
# y_train_list = []
# y_test_list = []
# for i in range(len(X_list)):
#     X = X_list[i]
#     y = y_list[i]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
    
#     X_train_list.append(X_train)
#     X_test_list.append(X_test)
#     y_train_list.append(y_train)
#     y_test_list.append(y_test)


In [ ]:
# # combine ages
# X_train = np.vstack((X_train_list))
# X_test = np.vstack((X_test_list))
# y_train = np.vstack((y_train_list))
# y_test = np.vstack((y_test_list))

In [ ]:
# with open(path + 'data_sampling/equal_age/x_train.npy', 'wb') as f:
#     np.save(f, X_train)

    
# with open(path + 'data_sampling/equal_age/x_test.npy', 'wb') as f:
#     np.save(f, X_test)
    
    
# with open(path + 'data_sampling/equal_age/y_train.npy', 'wb') as f:
#     np.save(f, y_train)
    
# with open(path + 'data_sampling/equal_age/y_test.npy', 'wb') as f:
#     np.save(f, y_test)

# all regressions

## Linear Regression

In [105]:
# Linear Regression
lm = LinearRegression()
lm.fit(X_train,y_train)

y_pred_lm = lm.predict(X_test) # predicting using train dataset

coeffs = lm.coef_
print(coeffs)

[[ 3.32493086e-04 -7.82741216e-03  2.11908608e+12  2.06433302e+11
  -3.11652039e+11 -1.02153808e+12  3.53930215e+11 -1.09998813e+12
  -3.23578423e+11 -9.53593880e+11  1.12157955e+12  2.67665275e+12
  -7.16215463e+11 -6.58997360e+10  9.71576877e-01]]


In [106]:
mse_lm = mean_squared_error(y_test, y_pred_lm)
print("MSE model_linear:", mse_lm)

MSE model_linear: 0.005773620952774034


## Ridge Regression

In [107]:
# Ridge Regression
ridge = Ridge(alpha= 0.5)
ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_test)
coeffs = ridge.coef_
print(coeffs)

[[ 1.61514650e-04 -7.09863023e-03 -5.65533871e-02  1.96451547e-02
   5.03103552e-01 -1.56247514e-01  2.99248848e-01 -4.49955227e-02
  -7.06602162e-01  2.12380099e-02 -1.70026339e-02 -6.67778668e-02
   5.42432652e-02  2.15010625e-01  9.71331973e-01]]


In [108]:
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print("MSE model_ridge:", mse_ridge)

MSE model_ridge: 0.005801168436719107


## Ridge Regression with Cross Validation--?

In [109]:
# Ridge Regression with CV and with polynomial degree 2 features

# Polynomial Fit

poly_train = PolynomialFeatures(2)
X_train_poly = poly_train.fit_transform(X_train)

poly_test = PolynomialFeatures(2)
X_test_poly = poly_test.fit_transform(X_test)

ridgeCV_poly = RidgeCV(cv=10).fit(X_train_poly,y_train)
y_pred_ridgeCV_poly = ridgeCV_poly.predict(X_test_poly)


In [110]:
mse_ridgeCV_poly = mean_squared_error(y_test, y_pred_ridgeCV_poly)
print("MSE model_ridgeCV_poly:", mse_ridgeCV_poly)

MSE model_ridgeCV_poly: 1.213749605505218e-06


## Decision tree

In [111]:
# Decision tree
dt = DecisionTreeRegressor(random_state=3).fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [112]:
mse_dt = mean_squared_error(y_test, y_pred_dt)
print("MSE model_dt:", mse_dt)

MSE model_dt: 3.321141146320967e-09


## Random Forest Regressor

In [113]:
# Random Forest Regressor
rf = RandomForestRegressor(max_depth=15, random_state=3)
rf.fit(X_train, y_train.reshape(len(y_train)))

y_pred_rf = rf.predict(X_test)


In [114]:
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("MSE model_rf:", mse_rf)

MSE model_rf: 4.338431274960168e-09
